In [ ]:
# import modules and define functions
from pyscf import gto, scf
import pandas as pd
import numpy as np
import os
from functools import reduce
from scipy import linalg
os.environ['OMP_NUM_THREADS'] = "1" 

: 

In [ ]:
# build H2 with minimal basis
atom = 'O 0 0 0; O 0 0 1.21'
basis = '6-31g'
# initiate a triplet O2
o2_triplet = gto.Mole()
o2_triplet.build(atom=atom,basis=basis,charge=0,spin=2)

In [ ]:
# calculate UHF and ROHF energies of the triplet O2
o2_0_3_uhf = scf.UHF(o2_triplet).run()
o2_0_3_rohf = scf.ROHF(o2_triplet).run()

In [ ]:
# Energy differences
delE_0_3_U_RO = (o2_0_3_uhf.e_tot - o2_0_3_rohf.e_tot) * 627.509
print('The UHF energy is lower than the ROHF energy by %s for O2_0_3' %(round(delE_0_3_U_RO, 2)))

In [ ]:
# S^2 values for various orbitals
print('The <S^2> value for UHF O2_0_3 is %.2f' %(o2_0_3_uhf.spin_square()[0]))
print('The <S^2> value for ROHF O2_0_3 is %.2f' %(o2_0_3_rohf.spin_square()[0]))

In [ ]:
# S is the overlap matrix among phi_a's and phi_b's of the size MxM, e.g. S12 = <phi_a_1|phi_b_2>
# M is the total number of basis functions, i.e. 8 BFs from O0 and 8 BFs from O1.
S = o2_0_3_uhf.get_ovlp()
# mo_coeff is the mo coefficients of the size 2xMxM. 2 is for alpha and beta orbitals.
mo_coeff = o2_0_3_uhf.mo_coeff
# mo_occ is the mo occupation matrix of the size 2xM. 2 is for alpha and beta orbitals.
mo_occ = o2_0_3_uhf.mo_occ

# define the range of summation, i.e. \Sigma_1_Na(\Sigma_1_Nb(c_ij_a.T*Sij*c_ij_b)
mo = (mo_coeff[0][:,mo_occ[0]>0], mo_coeff[1][:,mo_occ[1]>0])
# mo coefficiants for the occupied alpha and beta orbitals, respectively.
mo_a, mo_b = mo
# get the number of occupied orbitals for alpha and beta.
nocc_a = mo_a.shape[1]
nocc_b = mo_b.shape[1]
# obtain S^(alphabeta)_ij
s = reduce(np.dot, (mo_a.conj().T, S, mo_b))
# calcualate Nb - \Sigma_1_Na(\Sigma_1_Nb(c_ij_a.T*Sij*c_ij_b), or the amount of spin contamination
S2_corr = np.sum(mo_occ[1]) - np.sum(np.dot(s.T, s))
print(S2_corr)

In [ ]:
# orbital energy and occupations
o2_0_3_uhf.analyze(verbose=3)

In [ ]:
# orbital energy and occupations
o2_0_3_rohf.analyze(verbose=3)

In [ ]:
# orbital energy and occupations
o2_0_1_uhf.analyze(verbose=3)

In [ ]:
# orbital energy and occupations
o2_0_1_rohf.analyze(verbose=3)

In [ ]:
# orbital energy and occupations
o2_0_1_rhf.analyze(verbose=3)